This noteboook displays an attempt to recreate the Smith-Waterman algorithm for pairiwse local alignments using the BLOSUM62 matrix and a penalty for introducing gaps.

The Smith-Waterman algorithm is an alignment tool for pairwise local alignments. This is useful to compare protein sequences that are not necessarily homologous and differ in length and may for example identify important structural elements or domains.

Briefly explained, the algorithm creates a 'matrix' of two sequences. For each position in the alignment, there are three possibilities: There could be a gap in sequence A or in sequence B or neither. To identify the one that leads to the best result, the algorithm compares three numerical scores for each position. This score is the result of adding or substracting a specific value from the previous score, starting at 0. This could be a penalty for introducing a gap in A or B, corresponding to moving to the right or down in the matrix, or the score for the two amino acids, or moving diagonally down to the right. This score will be taken from the BLOSUM62 matrix.

After filling out the matrix, the final alignment can be obtained by starting at the position with the overall highest score and backtracking to the first 0.

One way to access the BLOSUM62 matrix is via importing the BioPython module and assigning the matrix to a variable for easier access. This variable will be a dictionary, with a tuple containing the two amino acids as keys and their scores as values.

In [1]:
from Bio.SubsMat import MatrixInfo
blosum62 = MatrixInfo.blosum62
import github

C:\Users\lionr\AppData\Roaming\Python\Python310\site-packages\Bio\SubsMat\__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


A function is useful to access and return the score for two amino acids A and B, which are taken as input arguments. Should there be a gap, substract the gap penalty.

In [2]:
def get_blosum_score(A,B,gp):
    """This function returns the BLOSUM62 score for a given pair of amino acids."""
    if A=="-" or B=="-":
        blosum_score = gp
    else:
        try:
            blosum_score = blosum62[(A,B)]            
        except:
            blosum_score = blosum62[(B,A)]
    return blosum_score

In order to find the highest score, another function can be used. All scores will be stored as values in a dictionary, therefore the function will take a dictionary as an input argument and return the key corresponding to the highest value.

In [3]:
def get_max_key(dictx):
    value_list = list(dictx.values())
    key_list = list(dictx.keys())
    max_score = max(value_list)
    pos = value_list.index(max_score)
    S_maxscore = key_list[pos]
    return S_maxscore

With all these functions it's time to create the algorithm.

In [4]:
def smith_waterman(gp,A,B):
    # take two amino acid sequences and the gap penalty as input arguments and convert them to the respective data type
    if not(isinstance(A,list)) or not(isinstance(B,list)):
        A = list(A.upper())
        B = list(B.upper())
    gp = int(gp)
    
    # insert a gap in A and B at index position 0
    A.insert(0,"-")
    B.insert(0,"-")
    
    # create index counters for A and B as well as empty dictionaries and lists to store the data
    aidx = 0
    bidx = 1
    Scores = {}
    Temp_scores = {}
    A_aligned = []
    B_aligned = []
    
    # create the first group of scores for aidx == 0 and bidx == 0 and reset the counters to 1 afterwards
    Scores["S00"] = [[0],[0,0],["-","-"]]
    while bidx<len(B):
        Scores[f"S{aidx}{bidx}"] = [[0],[0,bidx-1],["-",B[bidx]]]
        bidx+=1
    bidx = 0
    aidx = 1
    while aidx<len(A):
        Scores[f"S{aidx}{bidx}"] = [[0],[aidx-1,0],[A[aidx],"-"]]
        aidx+=1
    bidx=1
    aidx=1
            
    # fill out the matrix for the alignment by comparing all the temporary scores for each position
    # keep the one with the highest score
    while bidx<len(B):
        
        # reset aidx to 1
        aidx=1
        
        while aidx<len(A):
            # S1 - no gap: add or substract BLOSUM62 score for the amino acid pair to the previous score left diagonally above
            Temp_scores[f"S{aidx}{bidx}1"] = [Scores[f"S{aidx-1}{bidx-1}"][0][0] + get_blosum_score(A[aidx],B[bidx],gp)]
            if Temp_scores[f"S{aidx}{bidx}1"][0]<0:
                Temp_scores[f"S{aidx}{bidx}1"][0]==0
            # S2 - gap in A: substract gap penalty from previous score to the right
            Temp_scores[f"S{aidx}{bidx}2"] = [Scores[f"S{aidx}{bidx-1}"][0][0] + gp]
            if Temp_scores[f"S{aidx}{bidx}2"][0]<0:
                Temp_scores[f"S{aidx}{bidx}2"][0]==0
            # S3 - gap in B: substract gap penalty from previous score above
            Temp_scores[f"S{aidx}{bidx}3"] = [Scores[f"S{aidx-1}{bidx}"][0][0] + gp]
            if Temp_scores[f"S{aidx}{bidx}3"][0]<0:
                Temp_scores[f"S{aidx}{bidx}3"][0]==0
            
            # find the highest score and create a new key value pair in the dictionary with the score,
            # the position from which the score was calculated and the respecitve positions in the alignment
            if Temp_scores[f"S{aidx}{bidx}1"]>=Temp_scores[f"S{aidx}{bidx}2"] and Temp_scores[f"S{aidx}{bidx}1"]>=Temp_scores[f"S{aidx}{bidx}3"]:
                Scores[f"S{aidx}{bidx}"] = [Temp_scores[f"S{aidx}{bidx}1"],[aidx-1,bidx-1],[A[aidx],B[bidx]]]
            
            elif Temp_scores[f"S{aidx}{bidx}2"]>Temp_scores[f"S{aidx}{bidx}1"] and Temp_scores[f"S{aidx}{bidx}2"]>=Temp_scores[f"S{aidx}{bidx}3"]:
                Scores[f"S{aidx}{bidx}"] = [Temp_scores[f"S{aidx}{bidx}2"],[aidx,bidx-1],["-",B[bidx]]]

            elif Temp_scores[f"S{aidx}{bidx}3"]>Temp_scores[f"S{aidx}{bidx}1"] and Temp_scores[f"S{aidx}{bidx}3"]>Temp_scores[f"S{aidx}{bidx}2"]:
                Scores[f"S{aidx}{bidx}"] = [Temp_scores[f"S{aidx}{bidx}3"],[aidx-1,bidx],[A[aidx],"-"]]
                                            
            # repeat for all indeces in A
            aidx+=1

        # increment the counter for B and repeat for all remaining indeces in B                            
        bidx+=1
    
    # get the key for the highest numerical value in the dictionary as well as the index positions
    S_max = get_max_key(Scores)
    S_final = list(S_max)
    aidx = int(S_final[1])
    bidx = int(S_final[2])
    """print(Scores)
    print(S_max)"""
    
    if Scores[S_max][0][0]==0:
        fail = "No alignment possible."
        return fail
    
    # backtracking: create the local alignment
    while aidx>0 or bidx>0:
        new_score = Scores[f"S{aidx}{bidx}"][0][0]
        if new_score==0:
            break
        new_aidx = Scores[f"S{aidx}{bidx}"][1][0]
        new_bidx = Scores[f"S{aidx}{bidx}"][1][1]
        A_aligned.append(Scores[f"S{aidx}{bidx}"][2][0]) 
        B_aligned.append(Scores[f"S{aidx}{bidx}"][2][1])
        """print(A_aligned)
        print(B_aligned)"""
        aidx = new_aidx
        bidx = new_bidx
     
    # reverse the lists containing the final alignment
    A_aligned.reverse()
    B_aligned.reverse()
    
    # print the alignment and its score
    print(A_aligned)
    print(B_aligned)
    print(f"Alignment score using BLOSUM62 and a gap penalty of {gp} is {Scores[S_max][0][0]}.")

In [5]:
smith_waterman(-2,"design","deas")

['D', 'E', '-', 'S']
['D', 'E', 'A', 'S']
Alignment score using BLOSUM62 and a gap penalty of -2 is 13.


Put the algorithm to the test by creating random sequences and identifying the best local alignment.

In [6]:
import random
amino_acids = ["A","R","N","D","C","E","Q","G","H","I","L","K","M","F","P","S","T","W","Y","V"]

In [7]:
for i in range(6):
    test_A = random.choices(amino_acids,k=6)
    test_B = random.choices(amino_acids,k=3)
    print()
    print(test_A)
    print(test_B)
    smith_waterman(-2,test_A,test_B)


['M', 'M', 'L', 'H', 'N', 'W']
['E', 'L', 'N']
['N']
['N']
Alignment score using BLOSUM62 and a gap penalty of -2 is 6.

['A', 'D', 'P', 'F', 'Q', 'K']
['K', 'N', 'Y']
['K']
['K']
Alignment score using BLOSUM62 and a gap penalty of -2 is 5.

['R', 'T', 'F', 'G', 'G', 'D']
['I', 'S', 'F']
['-', 'T', 'F']
['I', 'S', 'F']
Alignment score using BLOSUM62 and a gap penalty of -2 is 5.

['R', 'Y', 'G', 'Q', 'H', 'Q']
['W', 'T', 'K']
['Y']
['W']
Alignment score using BLOSUM62 and a gap penalty of -2 is 2.

['N', 'Y', 'V', 'R', 'P', 'S']
['K', 'V', 'R']
['Y', 'V', 'R']
['K', 'V', 'R']
Alignment score using BLOSUM62 and a gap penalty of -2 is 7.

['N', 'G', 'F', 'V', 'Y', 'K']
['R', 'S', 'F']
['F']
['F']
Alignment score using BLOSUM62 and a gap penalty of -2 is 6.
